# Lab 2: Building a Code Generator with Few-Shot Learning

## Objectives
- Implement few-shot learning for code generation
- Create domain-specific code templates
- Build a code suggestion system
- Learn best practices for code generation prompts

## Prerequisites
- Completed Lab 1
- Understanding of basic prompt engineering concepts

In [ ]:
import os
import json
import openai
from typing import List, Dict, Optional
from dotenv import load_dotenv

load_dotenv()
client = openai.OpenAI(
    api_key=os.getenv('OPENAI_API_KEY')
)

## 1. Few-Shot Code Generation System

In [ ]:
class CodeExample:
    def __init__(self, description: str, code: str, explanation: str = ""):
        self.description = description
        self.code = code
        self.explanation = explanation
    
    def format(self) -> str:
        return f"Description: {self.description}\nCode:\n```python\n{self.code}\n```\nExplanation: {self.explanation}\n"

class FewShotCodeGenerator:
    def __init__(self):
        self.examples = self.load_examples()
    
    def load_examples(self) -> List[CodeExample]:
        # Pre-defined examples for few-shot learning
        return [
            CodeExample(
                "Create a function to calculate the factorial of a number",
                """def factorial(n: int) -> int:
    if n <= 1:
        return 1
    return n * factorial(n - 1)""",
                "A recursive implementation of factorial calculation"
            ),
            CodeExample(
                "Create a function to check if a string is a palindrome",
                """def is_palindrome(text: str) -> bool:
    text = text.lower()
    return text == text[::-1]""",
                "A simple palindrome checker using string slicing"
            )
        ]
    
    def generate_code(self, prompt: str, num_examples: int = 2) -> str:
        # Create the few-shot prompt
        few_shot_prompt = "Here are some examples of Python code generation:\n\n"
        
        # Add examples
        for example in self.examples[:num_examples]:
            few_shot_prompt += example.format() + "\n"
        
        # Add the new request
        few_shot_prompt += f"Now, please generate code for the following request:\n{prompt}"
        
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an expert Python programmer. Generate clean, efficient, and well-documented code."},
                {"role": "user", "content": few_shot_prompt}
            ],
            temperature=0.3
        )
        
        return response.choices[0].message.content

# Test the few-shot code generator
generator = FewShotCodeGenerator()
prompt = "Create a function to find the greatest common divisor of two numbers"
generated_code = generator.generate_code(prompt)
print(f"Generated code for: {prompt}\n")
print(generated_code)

## 2. Domain-Specific Code Generation

In [ ]:
class DomainSpecificCodeGenerator:
    def __init__(self, domain: str):
        self.domain = domain
        self.domain_examples = self.load_domain_examples()
    
    def load_domain_examples(self) -> Dict[str, List[CodeExample]]:
        return {
            "data_processing": [
                CodeExample(
                    "Load and preprocess a CSV file using pandas",
                    """import pandas as pd

def preprocess_data(file_path: str) -> pd.DataFrame:
    # Read CSV file
    df = pd.read_csv(file_path)
    
    # Handle missing values
    df = df.fillna(df.mean(numeric_only=True))
    
    # Convert categorical variables
    categorical_columns = df.select_dtypes(include=['object']).columns
    df = pd.get_dummies(df, columns=categorical_columns)
    
    return df""",
                    "A function to load and preprocess data using pandas"
                )
            ],
            "web_development": [
                CodeExample(
                    "Create a simple Flask API endpoint",
                    """from flask import Flask, jsonify, request

app = Flask(__name__)

@app.route('/api/data', methods=['POST'])
def process_data():
    data = request.json
    result = {'status': 'success', 'data': data}
    return jsonify(result)

if __name__ == '__main__':
    app.run(debug=True)""",
                    "A basic Flask API endpoint that accepts POST requests"
                )
            ]
        }
    
    def generate_code(self, prompt: str) -> str:
        if self.domain not in self.domain_examples:
            raise ValueError(f"Domain '{self.domain}' not supported")
        
        domain_prompt = f"Generate Python code for the following {self.domain} task:\n\n"
        
        # Add domain-specific examples
        for example in self.domain_examples[self.domain]:
            domain_prompt += example.format() + "\n"
        
        domain_prompt += f"Now, please generate code for:\n{prompt}"
        
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": f"You are an expert in {self.domain} using Python. Generate professional, well-documented code."},
                {"role": "user", "content": domain_prompt}
            ],
            temperature=0.3
        )
        
        return response.choices[0].message.content

# Test domain-specific generation
data_generator = DomainSpecificCodeGenerator("data_processing")
web_generator = DomainSpecificCodeGenerator("web_development")

data_prompt = "Create a function to normalize numerical columns in a DataFrame"
web_prompt = "Create a REST API endpoint for user registration"

print("Data Processing Example:")
print(data_generator.generate_code(data_prompt))
print("\nWeb Development Example:")
print(web_generator.generate_code(web_prompt))

## 3. Code Suggestion System

In [ ]:
class CodeSuggestionSystem:
    def __init__(self):
        self.context = ""
    
    def set_context(self, code_context: str):
        self.context = code_context
    
    def suggest_completion(self, partial_code: str, max_suggestions: int = 3) -> List[str]:
        prompt = f"""
        Given the following code context and partial code, suggest {max_suggestions} possible completions:
        
        Context:
        ```python
        {self.context}
        ```
        
        Partial code:
        ```python
        {partial_code}
        ```
        
        Provide {max_suggestions} different completion suggestions, each with a brief explanation.
        """
        
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an intelligent code completion system. Provide relevant and contextual code suggestions."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.5
        )
        
        return response.choices[0].message.content

# Test the code suggestion system
suggester = CodeSuggestionSystem()

# Set some context
context = """
class DataProcessor:
    def __init__(self, data):
        self.data = data
    
    def process(self):
        # Process the data
        pass
"""

suggester.set_context(context)

# Get suggestions for partial code
partial_code = "def validate_data(self, "
suggestions = suggester.suggest_completion(partial_code)
print("Code suggestions:")
print(suggestions)

## 4. Code Review and Improvement System

In [ ]:
class CodeReviewSystem:
    def __init__(self):
        self.review_criteria = [
            "Code style and PEP 8 compliance",
            "Documentation and comments",
            "Error handling",
            "Performance considerations",
            "Security best practices"
        ]
    
    def review_code(self, code: str) -> str:
        prompt = f"""
        Please review the following Python code considering these criteria:
        {', '.join(self.review_criteria)}
        
        Code to review:
        ```python
        {code}
        ```
        
        Provide:
        1. Specific issues and suggestions for improvement
        2. Code examples for suggested changes
        3. Best practices that could be applied
        """
        
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an experienced code reviewer. Provide detailed, constructive feedback for code improvement."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3
        )
        
        return response.choices[0].message.content

# Test the code review system
reviewer = CodeReviewSystem()

code_to_review = """
def process_data(d):
    result = []
    for i in range(len(d)):
        if d[i] > 0:
            result.append(d[i] * 2)
    return result
"""

review_feedback = reviewer.review_code(code_to_review)
print("Code Review Feedback:")
print(review_feedback)

## Exercises

1. Create a code generator for a new domain (e.g., machine learning, GUI development)
2. Implement a system that generates unit tests for given code
3. Build a code refactoring suggestion system
4. Create a documentation generator for Python code

## Next Steps
- Explore more advanced code generation techniques
- Implement support for multiple programming languages
- Add code analysis and optimization suggestions
- Create an interactive code learning system